In [1]:
%%bash

cd /DrakeDP
if test -f requirements.txt
    then
    sed -i '/jedi/d;/jupyter/d;' ./requirements.txt
    pip install -r ./requirements.txt "pyzmq<25.0.0"
    else echo "There's no requiremnts.txt, so nothing to install. This is the case with most projects."
fi


bash: line 2: cd: /DrakeDP: No such file or directory


There's no requiremnts.txt, so nothing to install. This is the case with most projects.


In [2]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LinearQuadraticRegulator,
    plot_system_graphviz,
    LogVectorOutput,
    PidController,
    MeshcatVisualizer,
    ModelVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
)



In [3]:
meshcat=StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [4]:
#Leafsystem or as symbolicvectorsystem dynamics as (xd=f(x))
builder=DiagramBuilder()

#instantiate double pendulum and scene_graph (container for geometries of all physical systems)
double_pendulum, scene_graph = AddMultibodyPlantSceneGraph(
    builder,
    time_step=0.0,
)
Parser(double_pendulum).AddModels("DoublePendulum.urdf") #AddModels() works for locally stored files
double_pendulum.Finalize()
double_pendulum.set_name("DoublePendulum") 

For this problem and URDF, the z axis is vertical and x axis horizontal with y axis going into the page. Theta1 at 0 is hanging down and theta 2 at 0 is upright.

In [5]:
xequilibrium = np.array([np.pi, 0, 0.0, 0.0])  # upright Equilibrium state: [theta1, theta2, theta1_dot, theta2_dot]

#Double check your URDF file is loaded correctly
with open("DoublePendulum.urdf", "r") as f:
    urdf_contents = f.read()
print(urdf_contents)

In [ ]:
def double_pendulum_balancing(xequilibrium, Q, R):
    context = double_pendulum.CreateDefaultContext()
    context.get_mutable_continuous_state_vector().SetFromVector(xequilibrium)  # Set the initial state to the equilibrium state for now
    double_pendulum.get_actuation_input_port().FixValue(context, np.zeros(2))  # Fix the input to zero for now, based on # of

    lqr = LinearQuadraticRegulator(
        double_pendulum,
        context,
        Q,
        R,
        input_port_index=double_pendulum.get_actuation_input_port().get_index(), 
    )

    lqr = builder.AddSystem(lqr)
    builder.Connect(double_pendulum.get_state_output_port(), lqr.get_input_port(0)) 
    builder.Connect(lqr.get_output_port(0), double_pendulum.get_actuation_input_port())

    actuation_logger = LogVectorOutput(lqr.get_output_port(), builder)
    state_logger = LogVectorOutput(double_pendulum.get_state_output_port(), builder)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    visualizer.set_name("visualizer")

    diagram = builder.Build()

    #how do you make this thing kill itself?--> you don't, every time builder=DiagramBuilder() is called a new diagram is created so you just rerun the cell. 
    #plot_system_graphviz(diagram)
    #Extract eoms
    return diagram, actuation_logger, state_logger


In [7]:
def simulate_and_animate(diagram, xstart, alogger, slogger, sim_time=5.0, visualize=True):
    simulator = Simulator(diagram)
    simulator.set_publish_every_time_step(False)
    simulator.get_mutable_integrator().set_fixed_step_mode(True)

    visualizer=diagram.GetSubsystemByName("visualizer")
    visualizer.StartRecording(False)

    if len(xstart) != diagram.GetSubsystemByName("DoublePendulum").num_continuous_states():
        print (f"Your plant doesn't have {len(xstart)} state variables.")
        return  
    
    # reset initial conditions
    context = simulator.get_mutable_context()
    context.SetTime(0.0)
    context.SetContinuousState(xstart)

    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)
    
    # stop video
    visualizer.PublishRecording()
    visualizer.DeleteRecording()

    # access logged data
    alog = alogger.FindLog(context)
    slog = slogger.FindLog(context)
    return alog, slog

In [8]:
# LQR for double pendulum balancing
Q = np.diag([10.0, 10.0, 1.0, 1.0])  # State cost matrix
R = np.eye(2)
xstart = np.array([np.pi, -np.pi/2, 0.0, 0.0])  # Initial state: [theta1, theta2, theta1_dot, theta2_dot]
diagram, actuation_logger, state_logger = double_pendulum_balancing(xequilibrium, Q, R)



In [9]:
a_log, s_log = simulate_and_animate(diagram, xstart, actuation_logger, state_logger, sim_time=6.0, visualize=True)
#log.sample_times()
u=a_log.data()
x=s_log.data()
print(u.shape)
print(x.shape)
for i in range(10):
    print(u[:,i])
    #print(x[:,-i])
     #looking at the logs max actuation is 32Nm (without real masses, but also no contact, which is why theta1 is high too.)
print(x[:,-1]) #initial state

(2, 385)
(4, 385)
[30.45044461 31.94282604]
[27.63716689 29.93937616]
[25.13623906 28.04316084]
[22.94645158 26.25212641]
[21.05692256 24.56143498]
[19.44983864 22.96434806]
[18.10281125 21.45302355]
[16.99080475 20.01919658]
[16.08764019 18.65473642]
[15.36710933 17.35208536]
[ 3.14158887e+00 -1.56880169e-06  8.55839952e-06  3.54492680e-06]
